In [3]:
from pyspark.sql import SparkSession

In [11]:
spark = SparkSession.builder.getOrCreate()
spark.conf.set('spark.sql.repl.eagerEval.enabled', True)
spark.conf.set('spark.sql.repl.eagerEval.maxNumRows',20)
spark.conf.set('spark.sql.execution.arrow.pyspark.enabled',True)

In [65]:
from datetime import datetime, date
import pandas as pd
from pyspark.sql import Row
import numpy as np
import pyspark.sql.functions as func
from pyspark.sql.functions import col
from pyspark.sql.types import FloatType
from pyspark.sql.types import DecimalType
from decimal import Decimal 

In [29]:
cursor = con.cursor()

In [30]:
sql = '''select cad1.midia, cad26.midia,cad12.midia,
cad1.investiment as cad1inv, cad1.impressions as cad1imp,
cad3.investiment as cad3inv, cad3.impressions as cad3imp,
cad12.investiment as cad12inv, cad12.impressions as cad12imp,
cad26.investiment as cad26inv, cad26.impressions as cad26imp, 
cad42.investiment as cad42inv, cad42.impressions as cad42imp
from (select midia,
		round(sum(cost),0) as investiment,
		round(sum(impressions),0) as impressions,
		date
		from datamarts.tb_consolidada_geral_dia tcgd
		where (date >= current_date-1)
		and (campaignname like '%cad1-%')
		and (midia in ('google','criteo','bing', 'facebook'))
		group by 1,date) cad1
full join
  (select midia,
		round(sum(cost),0) as investiment,
		round(sum(impressions),0) as impressions,
		date
		from datamarts.tb_consolidada_geral_dia tcgd
		where (date >= current_date-1)
		and (campaignname like '%cad3-%')
		and (midia in ('google','criteo','bing', 'facebook'))
		group by 1,date) cad3
on cad1.midia = cad3.midia
full join
  (select midia,
		round(sum(cost),0) as investiment,
		round(sum(impressions),0) as impressions,
		date
		from datamarts.tb_consolidada_geral_dia tcgd
		where (date >= current_date-1)
		and (campaignname like '%cad12-%')
		and (midia in ('google','criteo','bing', 'facebook'))
		group by 1,date) cad12
on cad12.midia = cad1.midia
full join
  (select midia,
		round(sum(cost),0) as investiment,
		round(sum(impressions),0) as impressions,
		date
		from datamarts.tb_consolidada_geral_dia tcgd
		where (date >= current_date-1)
		and (campaignname like '%cad26-%')
		and (midia in ('google','criteo','bing', 'facebook'))
		group by 1,date) cad26
on cad26.midia = cad1.midia
full join
  (select midia,
		round(sum(cost),0) as investiment,
		round(sum(impressions),0) as impressions,
		date
		from datamarts.tb_consolidada_geral_dia tcgd
		where (date >= current_date-1)
		and (campaignname like '%cad42-%')
		and (midia in ('google','criteo','bing', 'facebook'))
		group by 1,date) cad42
on cad42.midia = cad1.midia;'''

In [124]:
cursor.execute(sql)
colnames = [desc[0] for desc in cursor.description] 
df_cad42 = cursor.fetchall()

In [42]:
[Decimal(i) for i in df_cad42]

NameError: name 'Decimal' is not defined

In [99]:
[Decimal(i) for (i,j) in df_cad42 if type(j) == Decimal]

ValueError: too many values to unpack (expected 2)

In [95]:
type(df_cad42[0][3]) == Decimal

True

In [133]:
a = []
for i in df_cad42:
    for j in i:
        if(isinstance(j,Decimal)):
            a.append(str(j))
        else:
            a.append(j)

In [132]:
pd.DataFrame(list(a))

,0
0,bing
1,None
2,None
3,1878
4,56765
5,None
6,None
7,None
8,None
9,None


In [130]:
df = spark.createDataFrame(a,colnames)

TypeError: Can not infer schema for type: <class 'str'>

In [23]:
# cad42 = df.withColumn("investiment",df["investiment"].cast(DecimalType()))\
#             .withColumn("impressions",df["impressions"].cast(DecimalType()))

In [34]:
df

midia,midia,midia,cad1inv,cad1imp,cad3inv,cad3imp,cad12inv,cad12imp,cad26inv,cad26imp,cad42inv,cad42imp
bing,null,null,1878.000000000000...,56765.00000000000...,null,null,null,null,null,null,null,null
criteo,null,null,2239.000000000000...,552097.0000000000...,null,null,null,null,null,null,null,null
facebook,facebook,facebook,5558.000000000000...,408493.0000000000...,null,null,788.0000000000000...,72734.00000000000...,1820.000000000000...,173898.0000000000...,null,null
google,google,google,17369.00000000000...,2059404.000000000...,63884.00000000000...,2304087.000000000...,1207.000000000000...,13099.00000000000...,2002.000000000000...,1512216.000000000...,3533.000000000000...,84799.00000000000...


In [25]:
con.commit()

In [26]:
con.close()